In [1]:
import numpy as np
#from tools.tools import *
from myastrotools.reflectx import *
import astropy.units as u
import pandas as pandas

In [2]:
pl = pd.read_csv('../../ExploringGmagAOXTargetList/GMagAO-X-target-list.csv')
ind = [i for i in range(len(pl)) if '3021' in pl['pl_name'].loc[i]]
gj = pl.loc[ind]

In [ ]:
planet = 'GJ3021b'

## Planet:
Tint = 150 # Internal Temperature of your Planet in K
radius = gj['Re'].item() #Rearth
massj = gj['pl_msinij'].item() #msini
sma = gj['pl_orbsmax'].item()
ecc = gj['pl_orbeccen'].item()
sep_mean = sma * (1 + 0.5*ecc**2)

##### Get phase at average separation:
inc = gj['pl_orbincl'].item() # deg, keplerian_to_cartesian takes angles in degrees
if np.isnan(inc):
    inc = 60 #def
ecc = gj['pl_orbeccen'].item()
sma = gj['pl_orbsmax'].item()*u.au
argp = gj['pl_orblper'].item() # deg
lon = 0
star_mass = gj['st_mass'].item()
pl_mass = massj
distance = gj['sy_dist'].item()
from myastrotools.reflectx_supplemental import (GetPhaseAngle, keplerian_to_cartesian, keplersconstant, 
                                             danby_solve, eccentricity_anomaly)

kep = keplersconstant(star_mass*u.Msun,pl_mass*u.Mjup)
n = 100
meananom = np.linspace(0,2*np.pi,n)
seps = []
alphas = []
for j in range(len(meananom)):
    if np.isnan(argp):
        seps2 = []
        argps = []
        for w in np.linspace(0,2*np.pi, 50):
            pos, vel, acc = keplerian_to_cartesian(sma,ecc,inc,w,lon,meananom[j],kep)
            sep = np.sqrt(pos[0]**2,pos[1]**2)
            seps2.append(sep.value)
            argps.append(w)
        argp = np.mean(argps)
        sep = np.mean(seps2)*u.au
        alpha = GetPhaseAngle(meananom[j], ecc, inc, argp)

    else:
        pos, vel, acc = keplerian_to_cartesian(sma,ecc,inc,
                                               argp,lon,meananom[j],kep)
        sep = np.sqrt(pos[0]**2,pos[1]**2)
        alpha = GetPhaseAngle(meananom[j], ecc, inc, argp)
    seps.append(sep.value)
    alphas.append(alpha)
alphas = np.array(alphas)
seps = np.array(seps)

args = np.argsort(np.abs(seps - sep_mean))
phase_at_average_separation = np.min(alphas[args[:2]])


## Star:
T_star = gj['st_teff'].item() # K, star effective temperature
logg = gj['st_logg'].item() #logg for a 5000K star
metal = gj['st_met'].item() # solar metallicity
r_star = gj['st_rad'].item() # solar radius
Teq = ComputeTeq(T_star, r_star*u.Rsun, sep_mean*u.au, 
                 Ab = 0.3, fprime = 1/4) # planet equilibrium temperature 


## Climate:
nlevel = 91 # number of plane-parallel levels in your code
nofczns = 1 # number of convective zones initially. Let's not play with this for now.
nstr_upper = 85 # top most level of guessed convective zone
nstr_deep = nlevel -2 # this is always the case. Dont change this
nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convective zones
rfacv = 0.5

## Opacities:
#
planet_mh = 1 # Super Jupiter metallicity
planet_mh_str = '+000'
planet_mh_CtoO = 1.0

Teq_str = np.round(Teq, decimals=0)
sep_mean_str = np.round(sep_mean, decimals=2)
radius_str = np.round(radius, decimals=2)
directory = f'{planet}-Tstar{int(T_star)}-Rstar{r_star}-Teq{int(Teq_str)}-sep{sep_mean_str}-rad{radius_str}-mass{massj}-mh{int(planet_mh)}-co{planet_mh_CtoO}-phase{int(phase_at_average_separation)}'
#savefiledirectory = p['output_dir']+directory
output_dir = ''
savefiledirectory = output_dir+directory

local_ck_path = f'/Volumes/Oy/picaso/reference/kcoeff_2020/'
#local_ck_path = p['local_ck_path']

planet_properties = {
    'tint':Tint, 'Teq':Teq, 'radius':radius, 'radius_unit':u.Rjup,
     'mass':massj, 'mass_unit': u.Mjup,
     'gravity': None, 'gravity_unit':None,
    'semi_major':sep_mean, 'semi_major_unit': u.AU,
    'mh': planet_mh, 'CtoO':planet_mh_CtoO, 'phase':phase_at_average_separation, 'num_tangle':6,
    'num_gangle':6, 'noTiOVO':True, 'planet_mh_str':planet_mh_str,
    'local_ck_path':local_ck_path, 'distance':distance
}

star_properties = {
    'Teff':T_star, 'logg':logg, 'mh':metal, 'radius':r_star
}

climate_run_setup = {'climate_pbottom':2,
        'climate_ptop':-6,
        'nlevel':nlevel, 'nofczns':nofczns, 'nstr_upper':nstr_upper,
        'nstr_deep':nstr_deep, 'rfacv':rfacv
}
#opa_file = p['opa_file']
#opa_file = '/Volumes/Oy/picaso/reference/references/opacities/all_opacities_0.3_2_R60000.db'
opa_file = None
wave_range = [0.4,2.0]
R = 500
spectrum_setup = {'opacity_db':opa_file,
                  'wave_range':wave_range,
                  'calculation':'reflected', 'R':R
                 }

use_guillotpt = True

from myastrotools.reflectx import MakeModelCloudFreePlanet
cj = MakeModelCloudFreePlanet(planet_properties, 
                        star_properties,
                        use_guillotpt = use_guillotpt,
                        cdict = climate_run_setup,
                        compute_spectrum = True,
                        specdict = spectrum_setup,
                        savefiledirectory = savefiledirectory
             )

In [ ]:
fsed = OneCloudp['fsed']
logkzz = np.log10(OneCloudp['kzz'])
cloud_filename_prefix = f'cloudy-models/cloudy-fsed{fsed}-kzz{int(logkzz)}'
#cloud_filename_prefix = f'cloudy-fsed{fsed}-kzz{int(logkzz)}'
os.system('mkdir '+savefiledirectory+'/cloudy-models/')


### Cloud setup:
clouds_setup = {'kz':OneCloudp['kzz'], 
                'fsed':OneCloudp['fsed'], 
                'mean_mol_weight':OneCloudp['mmw'],
                #'condensates':OneCloudp['condensates'], 
                'condensates':['H2O', 'KCl'],
                'virga_mieff':OneCloudp['virga_mieff']
               }

clouds, clouds_added = MakeModelCloudyPlanet(savefiledirectory, 
                                                 clouds_setup, 
                                                 spectrum_setup,
                                                 cloud_filename_prefix,
                                                 calculation = 'planet')

In [1]:
pl = pd.read_csv('../../ExploringGmagAOXTargetList/GMagAO-X-target-list.csv')
ind = [i for i in range(len(pl)) if '40979' in pl['pl_name'].loc[i]]
gj = pl.loc[ind]
gj

,loc_rowid,pl_name,hostname,hd_name,hip_name,gaia_id,default_flag,sy_snum,sy_pnum,discoverymethod,...,Mam Inferred Teff from SpT,Sep arcsec,Sep lod,Re,SpT,StarTeff,Mam Inferred StarRad from Teff,StarRad,Teq,Giant
342,2856,HD 40979 b,HD 40979,HD 40979,HIP 28767,Gaia DR2 961428192989499904,0,3,1,Radial Velocity,...,NaN,0.025082,3.86016,11.967918,4.136501,6139.0,NaN,1.24,326.086806,False


In [ ]:
from myastrotools.reflectx import *
planet = 'HD40979b'

## Planet:
Tint = 150 # Internal Temperature of your Planet in K
radius = gj['Re'].item() #Rearth
massj = gj['pl_msinij'].item() #msini
sma = gj['pl_orbsmax'].item()
ecc = gj['pl_orbeccen'].item()
sep_mean = sma * (1 + 0.5*ecc**2)

##### Get phase at average separation:
inc = gj['pl_orbincl'].item() # deg, keplerian_to_cartesian takes angles in degrees
if np.isnan(inc):
    inc = 60 #def
ecc = gj['pl_orbeccen'].item()
sma = gj['pl_orbsmax'].item()*u.au
argp = gj['pl_orblper'].item() # deg
lon = 0
star_mass = gj['st_mass'].item()
pl_mass = massj
distance = gj['sy_dist'].item()
from myastrotools.reflectx_supplemental import (GetPhaseAngle, keplerian_to_cartesian, keplersconstant, 
                                             danby_solve, eccentricity_anomaly)

kep = keplersconstant(star_mass*u.Msun,pl_mass*u.Mjup)
n = 100
meananom = np.linspace(0,2*np.pi,n)
seps = []
alphas = []
for j in range(len(meananom)):
    if np.isnan(argp):
        seps2 = []
        argps = []
        for w in np.linspace(0,2*np.pi, 50):
            pos, vel, acc = keplerian_to_cartesian(sma,ecc,inc,w,lon,meananom[j],kep)
            sep = np.sqrt(pos[0]**2,pos[1]**2)
            seps2.append(sep.value)
            argps.append(w)
        argp = np.mean(argps)
        sep = np.mean(seps2)*u.au
        alpha = GetPhaseAngle(meananom[j], ecc, inc, argp)

    else:
        pos, vel, acc = keplerian_to_cartesian(sma,ecc,inc,
                                               argp,lon,meananom[j],kep)
        sep = np.sqrt(pos[0]**2,pos[1]**2)
        alpha = GetPhaseAngle(meananom[j], ecc, inc, argp)
    seps.append(sep.value)
    alphas.append(alpha)
alphas = np.array(alphas)
seps = np.array(seps)

args = np.argsort(np.abs(seps - sep_mean))
phase_at_average_separation = np.min(alphas[args[:2]])


## Star:
T_star = gj['st_teff'].item() # K, star effective temperature
logg = gj['st_logg'].item() #logg for a 5000K star
metal = gj['st_met'].item() # solar metallicity
r_star = gj['st_rad'].item() # solar radius
Teq = ComputeTeq(T_star, r_star*u.Rsun, sep_mean*u.au, 
                 Ab = 0.3, fprime = 1/4) # planet equilibrium temperature 


## Climate:
nlevel = 91 # number of plane-parallel levels in your code
nofczns = 1 # number of convective zones initially. Let's not play with this for now.
nstr_upper = 85 # top most level of guessed convective zone
nstr_deep = nlevel -2 # this is always the case. Dont change this
nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convective zones
rfacv = 0.5

## Opacities:
#
planet_mh = 1 # Super Jupiter metallicity
planet_mh_str = '+000'
planet_mh_CtoO = 1.0

Teq_str = np.round(Teq, decimals=0)
sep_mean_str = np.round(sep_mean, decimals=2)
radius_str = np.round(radius, decimals=2)
directory = f'{planet}-Tstar{int(T_star)}-Rstar{r_star}-Teq{int(Teq_str)}-sep{sep_mean_str}-rad{radius_str}-mass{massj}-mh{int(planet_mh)}-co{planet_mh_CtoO}-phase{int(phase_at_average_separation)}'
#savefiledirectory = p['output_dir']+directory
output_dir = ''
savefiledirectory = output_dir+directory

local_ck_path = f'/Volumes/Oy/picaso/reference/kcoeff_2020/'
#local_ck_path = p['local_ck_path']

planet_properties = {
    'tint':Tint, 'Teq':Teq, 'radius':radius, 'radius_unit':u.Rjup,
     'mass':massj, 'mass_unit': u.Mjup,
     'gravity': None, 'gravity_unit':None,
    'semi_major':sep_mean, 'semi_major_unit': u.AU,
    'mh': planet_mh, 'CtoO':planet_mh_CtoO, 'phase':phase_at_average_separation, 'num_tangle':6,
    'num_gangle':6, 'noTiOVO':True, 'planet_mh_str':planet_mh_str,
    'local_ck_path':local_ck_path, 'distance':distance
}

star_properties = {
    'Teff':T_star, 'logg':logg, 'mh':metal, 'radius':r_star
}

climate_run_setup = {'climate_pbottom':2,
        'climate_ptop':-6,
        'nlevel':nlevel, 'nofczns':nofczns, 'nstr_upper':nstr_upper,
        'nstr_deep':nstr_deep, 'rfacv':rfacv
}
#opa_file = p['opa_file']
#opa_file = '/Volumes/Oy/picaso/reference/references/opacities/all_opacities_0.3_2_R60000.db'
opa_file = None
wave_range = [0.4,2.0]
R = 500
spectrum_setup = {'opacity_db':opa_file,
                  'wave_range':wave_range,
                  'calculation':'reflected', 'R':R
                 }

use_guillotpt = True

from myastrotools.reflectx import MakeModelCloudFreePlanet
cj = MakeModelCloudFreePlanet(planet_properties, 
                        star_properties,
                        use_guillotpt = use_guillotpt,
                        cdict = climate_run_setup,
                        compute_spectrum = True,
                        specdict = spectrum_setup,
                        savefiledirectory = savefiledirectory
             )

In [2]:
pl = pd.read_csv('../../ExploringGmagAOXTargetList/GMagAO-X-target-list.csv')
ind = [i for i in range(len(pl)) if '183263 b' in pl['pl_name'].loc[i]]
gj = pl.loc[ind]
gj

,loc_rowid,pl_name,hostname,hd_name,hip_name,gaia_id,default_flag,sy_snum,sy_pnum,discoverymethod,...,Mam Inferred Teff from SpT,Sep arcsec,Sep lod,Re,SpT,StarTeff,Mam Inferred StarRad from Teff,StarRad,Teq,Giant
216,2220,HD 183263 b,HD 183263,HD 183263,HIP 95740,Gaia DR2 4296208099222829568,0,1,2,Radial Velocity,...,NaN,0.027825,4.282354,11.962777,4.305213,5875.99,NaN,1.29,239.708283,False


In [ ]:
from myastrotools.reflectx import *
planet = 'HD183263b'

## Planet:
Tint = 150 # Internal Temperature of your Planet in K
radius = gj['Re'].item() #Rearth
massj = gj['pl_msinij'].item() #msini
sma = gj['pl_orbsmax'].item()
ecc = gj['pl_orbeccen'].item()
sep_mean = sma * (1 + 0.5*ecc**2)

##### Get phase at average separation:
inc = gj['pl_orbincl'].item() # deg, keplerian_to_cartesian takes angles in degrees
if np.isnan(inc):
    inc = 60 #def
ecc = gj['pl_orbeccen'].item()
sma = gj['pl_orbsmax'].item()*u.au
argp = gj['pl_orblper'].item() # deg
lon = 0
star_mass = gj['st_mass'].item()
pl_mass = massj
distance = gj['sy_dist'].item()
from myastrotools.reflectx_supplemental import (GetPhaseAngle, keplerian_to_cartesian, keplersconstant, 
                                             danby_solve, eccentricity_anomaly)

kep = keplersconstant(star_mass*u.Msun,pl_mass*u.Mjup)
n = 100
meananom = np.linspace(0,2*np.pi,n)
seps = []
alphas = []
for j in range(len(meananom)):
    if np.isnan(argp):
        seps2 = []
        argps = []
        for w in np.linspace(0,2*np.pi, 50):
            pos, vel, acc = keplerian_to_cartesian(sma,ecc,inc,w,lon,meananom[j],kep)
            sep = np.sqrt(pos[0]**2,pos[1]**2)
            seps2.append(sep.value)
            argps.append(w)
        argp = np.mean(argps)
        sep = np.mean(seps2)*u.au
        alpha = GetPhaseAngle(meananom[j], ecc, inc, argp)

    else:
        pos, vel, acc = keplerian_to_cartesian(sma,ecc,inc,
                                               argp,lon,meananom[j],kep)
        sep = np.sqrt(pos[0]**2,pos[1]**2)
        alpha = GetPhaseAngle(meananom[j], ecc, inc, argp)
    seps.append(sep.value)
    alphas.append(alpha)
alphas = np.array(alphas)
seps = np.array(seps)

args = np.argsort(np.abs(seps - sep_mean))
phase_at_average_separation = np.min(alphas[args[:2]])


## Star:
T_star = gj['st_teff'].item() # K, star effective temperature
logg = gj['st_logg'].item() #logg for a 5000K star
metal = gj['st_met'].item() # solar metallicity
r_star = gj['st_rad'].item() # solar radius
Teq = ComputeTeq(T_star, r_star*u.Rsun, sep_mean*u.au, 
                 Ab = 0.3, fprime = 1/4) # planet equilibrium temperature 


## Climate:
nlevel = 91 # number of plane-parallel levels in your code
nofczns = 1 # number of convective zones initially. Let's not play with this for now.
nstr_upper = 85 # top most level of guessed convective zone
nstr_deep = nlevel -2 # this is always the case. Dont change this
nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convective zones
rfacv = 0.5

## Opacities:
#
planet_mh = 1 # Super Jupiter metallicity
planet_mh_str = '+000'
planet_mh_CtoO = 1.0

Teq_str = np.round(Teq, decimals=0)
sep_mean_str = np.round(sep_mean, decimals=2)
radius_str = np.round(radius, decimals=2)
directory = f'{planet}-Tstar{int(T_star)}-Rstar{r_star}-Teq{int(Teq_str)}-sep{sep_mean_str}-rad{radius_str}-mass{massj}-mh{int(planet_mh)}-co{planet_mh_CtoO}-phase{int(phase_at_average_separation)}'
#savefiledirectory = p['output_dir']+directory
output_dir = ''
savefiledirectory = output_dir+directory

local_ck_path = f'/Volumes/Oy/picaso/reference/kcoeff_2020/'
#local_ck_path = p['local_ck_path']

planet_properties = {
    'tint':Tint, 'Teq':Teq, 'radius':radius, 'radius_unit':u.Rjup,
     'mass':massj, 'mass_unit': u.Mjup,
     'gravity': None, 'gravity_unit':None,
    'semi_major':sep_mean, 'semi_major_unit': u.AU,
    'mh': planet_mh, 'CtoO':planet_mh_CtoO, 'phase':phase_at_average_separation, 'num_tangle':6,
    'num_gangle':6, 'noTiOVO':True, 'planet_mh_str':planet_mh_str,
    'local_ck_path':local_ck_path, 'distance':distance
}

star_properties = {
    'Teff':T_star, 'logg':logg, 'mh':metal, 'radius':r_star
}

climate_run_setup = {'climate_pbottom':2,
        'climate_ptop':-6,
        'nlevel':nlevel, 'nofczns':nofczns, 'nstr_upper':nstr_upper,
        'nstr_deep':nstr_deep, 'rfacv':rfacv
}
#opa_file = p['opa_file']
#opa_file = '/Volumes/Oy/picaso/reference/references/opacities/all_opacities_0.3_2_R60000.db'
opa_file = None
wave_range = [0.4,2.0]
R = 500
spectrum_setup = {'opacity_db':opa_file,
                  'wave_range':wave_range,
                  'calculation':'reflected', 'R':R
                 }

use_guillotpt = True

from myastrotools.reflectx import MakeModelCloudFreePlanet
cj = MakeModelCloudFreePlanet(planet_properties, 
                        star_properties,
                        use_guillotpt = use_guillotpt,
                        cdict = climate_run_setup,
                        compute_spectrum = True,
                        specdict = spectrum_setup,
                        savefiledirectory = savefiledirectory
             )